In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import preprocess as datapre

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import log_loss, make_scorer
from sklearn.preprocessing import LabelEncoder

In [2]:
train_data = pd.read_csv("../datasets/train_preprocess.csv")

In [3]:
train_data = datapre.dataset_sample(train_data)

In [4]:
train_data["DayOfWeekID"].dtype

dtype('int64')

In [5]:
feature_names = ['Year', 'Month', 'Hour', 'DayOfWeekID', 'PdDistrictID', \
                 'HasBlock', 'RoadTypeID', 'RoadBlockID', 'RoadName1ID', 'RoadName2ID', 'X', 'Y']

In [6]:
X = train_data[feature_names]

In [7]:
TargetEnc = LabelEncoder()
y_true = TargetEnc.fit_transform(train_data["Category"])

In [8]:
xgbclf = xgb.XGBClassifier(max_depth=6,
                       learning_rate=0.1, 
                       n_estimators=100, 
                       objective="multi:softprob",
                       n_job=-1,
                       gamma=2,
                       min_child_weight=10,
                       max_delta_step=2,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       random_state=10
                    )

In [ ]:
param_grid = {"n_estimators": list(range(400, 2001, 400))}

cv_clf_n_estimators = GridSearchCV(estimator=xgbclf, 
                                  param_grid = param_grid,
                                  scoring = make_scorer(log_loss, needs_proba=True), n_jobs=-1,
                                  cv=5, verbose=4, return_train_score=True, refit=False)

from time import time
start = time()
cv_clf_n_estimators.fit(X, y_true)
print("Training with total data elapse %d sencond." % (time()-start))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=400 ................................................
[CV] n_estimators=400 ................................................
[CV] n_estimators=400 ................................................
[CV] n_estimators=400 ................................................
